# Setup for google colab

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd "/content/drive/MyDrive/ColabNotebooks/Audio-ex3"

In [ ]:
!git pull

In [ ]:
!pip install -r requirements.txt
%cd src

---
# Check if GPU is available

In [2]:
import torch
torch.cuda.is_available()

True

---
# Logger

In [3]:
from lightning.pytorch.loggers import NeptuneLogger
from dotenv import load_dotenv
import os

load_dotenv('./../.env')
API_TOKEN = os.environ.get("API_TOKEN")
PROJECT_NAME = os.environ.get("PROJECT_NAME")

---
# Loading Data

In [4]:
import importlib
imported_module = importlib.import_module("loader")
importlib.reload(imported_module)
import loader
data_loader = loader.load_data()

# Create the classifier and train it

In [6]:
import importlib
importlib.reload(importlib.import_module("classifier"))
importlib.reload(importlib.import_module("loader"))
import classifier
import lightning as L
import loader


neptune_logger = NeptuneLogger(project=PROJECT_NAME, api_key=API_TOKEN, log_model_checkpoints=False)
model = classifier.NeuralNetwork()
trainer = L.Trainer(max_epochs=4, logger=neptune_logger, accelerator="auto", devices="auto", strategy="auto")
trainer.fit(model, data_loader['train'])
#after training, save the model
trainer.save_checkpoint("model.pth")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mrobay/Audio-ex3/e/AUD-27


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | lstm  | LSTM    | 604 K  | train
1 | cnv   | Conv2d  | 10.0 K | train
2 | conv1 | Conv2d  | 1.0 K  | train
3 | relu  | ReLU    | 0      | train
4 | loss  | CTCLoss | 0      | train
------------------------------------------
615 K     Trainable params
0         Non-trainable params
615 K     Total params
2.460     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

TypeError: decode_digit() missing 1 required positional argument: 'loss_fn'

# Test the classifier

In [3]:
trainer.test(model, data_loader['test'])

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000029AE841FEC0>
Traceback (most recent call last):
  File "C:\Users\merzi\PycharmProjects\pythonProject\Audio-ex3\venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "C:\Users\merzi\PycharmProjects\pythonProject\Audio-ex3\venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1438, in _shutdown_workers
    self._mark_worker_as_unavailable(worker_id, shutdown=True)
  File "C:\Users\merzi\PycharmProjects\pythonProject\Audio-ex3\venv\Lib\site-packages\torch\utils\data\dataloader.py", line 1380, in _mark_worker_as_unavailable
    assert self._workers_status[worker_id] or (self._persistent_workers and shutdown)
           ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
